# MLP para Predição de Indicadores Sociais

### Importação das bibliotecas

In [1]:
import pandas as pd
import numpy as np
import xlrd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


### Extração dos Arquivos

In [2]:

arquivo = xlrd.open_workbook('datasets/Tabelas1a21.xls')

planilhas_alvo = ['tab2', 'tab4', 'tab6', 'tab8', 'tab19']
abas = {}

for nome in planilhas_alvo:
    abas[nome] = arquivo.sheet_by_name(nome)
    
capitais = [
    'Porto Velho', 'Rio Branco', 'Manaus', 'Boa Vista', 'Belém', 'Macapá', 'Palmas',
    'São Luís', 'Teresina', 'Fortaleza', 'Natal', 'João Pessoa', 'Recife', 'Maceió', 
    'Aracaju', 'Salvador', 'Belo Horizonte', 'Vitória', 'Rio de Janeiro', 'São Paulo',
    'Curitiba', 'Florianópolis', 'Porto Alegre', 'Campo Grande', 'Cuiabá', 'Goiânia', 'Brasília'
]

capitais_treinamento = capitais[:20]
capitais_teste = capitais[20:]

print(f"Capitais para treino ({len(capitais_treinamento)}): {capitais_treinamento}")
print(f"Capitais para teste ({len(capitais_teste)}): {capitais_teste}")


Capitais para treino (20): ['Porto Velho', 'Rio Branco', 'Manaus', 'Boa Vista', 'Belém', 'Macapá', 'Palmas', 'São Luís', 'Teresina', 'Fortaleza', 'Natal', 'João Pessoa', 'Recife', 'Maceió', 'Aracaju', 'Salvador', 'Belo Horizonte', 'Vitória', 'Rio de Janeiro', 'São Paulo']
Capitais para teste (7): ['Curitiba', 'Florianópolis', 'Porto Alegre', 'Campo Grande', 'Cuiabá', 'Goiânia', 'Brasília']


Tabela 2 (Domicílios por sexo - 2000)

In [3]:
def extrair_tabela2():
    aba2 = abas['tab2']
    dados = {}
    
    # achar ano 2000
    start_linha = None
    for i in range(aba2.nrows):
        if '2000' in str(aba2.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 2")
    
    for i in range(start_linha, aba2.nrows):
        capital = str(aba2.cell_value(i, 0)).strip()
        if capital in capitais_treinamento:
            perc_mulheres = aba2.cell_value(i, 5)
            dados[capital] = perc_mulheres
    
    return dados

Tabela 4 (Mulheres +15 anos responsáveis por domicílios - 2000)

In [4]:
def extrair_tabela4():
    aba4 = abas['tab4']
    dados = {}

    start_linha = None
    for i in range(aba4.nrows):
        if '2000' in str(aba4.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 4")
    
    for i in range(start_linha, aba4.nrows):
        capital = str(aba4.cell_value(i, 0)).strip()
        if capital in capitais_treinamento:
            perc_25_29 = aba4.cell_value(i, 4)
            perc_35_39 = aba4.cell_value(i, 6)
            perc_45_49 = aba4.cell_value(i, 8)
            dados[capital] = {
                'perc_25_29': perc_25_29,
                'perc_35_39': perc_35_39,
                'perc_45_49': perc_45_49
            }
    
    return dados

Tabela 6 (Responsáveis por domicílios, por média de anos de estudo e sexo - 2000)

In [5]:
def extrair_tabela6():
    aba6 = abas['tab6']
    dados = {}
    
    start_linha = None
    for i in range(aba6.nrows):
        if '2000' in str(aba6.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 6")
    
    for i in range(start_linha, aba6.nrows):
        capital = str(aba6.cell_value(i, 0)).strip() 
        if capital in capitais_treinamento:
            anos_estudo = aba6.cell_value(i, 1)
            dados[capital] = anos_estudo
    
    return dados

Tabela 8 (Rendimento mensal médio, mulheres 10+ anos - 2000)

In [6]:
def extrair_tabela8():
    aba8 = abas['tab8']
    dados = {}
    
    start_linha = None
    for i in range(aba8.nrows):
        if '2000' in str(aba8.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 8")
    
    for i in range(start_linha, aba8.nrows):
        capital = str(aba8.cell_value(i, 0)).strip()
        if capital in capitais_treinamento:
            rendimento = aba8.cell_value(i, 2)
            dados[capital] = rendimento
    
    return dados

Tabela 19 (Proporção população 10+ anos por alfabetização e sexo - 2000)

In [7]:
def extrair_tabela19():
    aba19 = abas['tab19']
    dados = {}
    
    start_linha = None
    for i in range(aba19.nrows):
        if '2000' in str(aba19.cell_value(i, 0)):
            start_linha = i + 1
            break
    
    if start_linha is None:
        raise ValueError("Não encontrou dados de 2000 na Tabela 19")
    
    for i in range(start_linha, aba19.nrows):
        capital = str(aba19.cell_value(i, 0)).strip()
        if capital in capitais_treinamento:
            try:
                perc_criancas = aba19.cell_value(i, 3)
                dados[capital] = perc_criancas
            except:
                for col in range(1, aba19.ncols):
                    try:
                        valor = aba19.cell_value(i, col)
                        if isinstance(valor, (int, float)) and 0 <= valor <= 100:
                            perc_criancas = valor
                            dados[capital] = perc_criancas
                            break
                    except:
                        continue
    
    return dados

Extraindo

In [8]:
dados_tabela2 = extrair_tabela2()
dados_tabela4 = extrair_tabela4()
dados_tabela6 = extrair_tabela6()
dados_tabela8 = extrair_tabela8()
dados_tabela19 = extrair_tabela19()

Testando se pegou tudo

In [9]:
print("Dados da Tabela 2:", dados_tabela2)
print("Dados da Tabela 4:", dados_tabela4)
print("Dados da Tabela 6:", dados_tabela6)
print("Dados da Tabela 8:", dados_tabela8)
print("Dados da Tabela 19:", dados_tabela19)

Dados da Tabela 2: {'Porto Velho': 28.92975789297579, 'Rio Branco': 31.23068759948812, 'Manaus': 32.05793447799004, 'Boa Vista': 28.06391401554776, 'Belém': 35.37516196954972, 'Macapá': 33.269867549668874, 'Palmas': 23.38003252774845, 'São Luís': 34.10406910908812, 'Teresina': 31.41702646506176, 'Fortaleza': 33.205659226085814, 'Natal': 30.273985701670014, 'João Pessoa': 31.743983142922993, 'Recife': 37.40233284222732, 'Maceió': 32.3129762584237, 'Aracaju': 35.344376933558436, 'Salvador': 37.458409655869175, 'Belo Horizonte': 33.0116939057709, 'Vitória': 33.84137076603006, 'Rio de Janeiro': 35.33847810660211, 'São Paulo': 29.052869462825736}
Dados da Tabela 4: {'Porto Velho': {'perc_25_29': 10.144987401379653, 'perc_35_39': 14.74658184972531, 'perc_45_49': 11.198314676360031}, 'Rio Branco': {'perc_25_29': 10.123925644613232, 'perc_35_39': 12.617429542274635, 'perc_45_49': 10.488706775934439}, 'Manaus': {'perc_25_29': 10.030348724017484, 'perc_35_39': 13.415472123074574, 'perc_45_49': 1

DataFrame dos Dados Extraídos

In [12]:
dataset = []

for capital in capitais_treinamento:
    if (capital in dados_tabela2 and 
        capital in dados_tabela4 and 
        capital in dados_tabela6 and 
        capital in dados_tabela8 and 
        capital in dados_tabela19):
        
        tab2_val = dados_tabela2[capital]
        tab4_val = dados_tabela4[capital]
        tab6_val = dados_tabela6[capital]
        tab8_val = dados_tabela8[capital]
        tab19_val = dados_tabela19[capital]
        
        linha = {
            'capital': capital,
            'perc_mulheres': tab2_val,
            'perc_25_29': tab4_val['perc_25_29'],
            'perc_35_39': tab4_val['perc_35_39'],
            'perc_45_49': tab4_val['perc_45_49'],
            'anos_estudo': tab6_val,
            'rendimento': tab8_val,
            'alfabetizacao': tab19_val
        }
        dataset.append(linha)

df = pd.DataFrame(dataset)
print(f"Capitais de treinamento:")
df.head(27)

Capitais de treinamento:


,capital,perc_mulheres,perc_25_29,perc_35_39,perc_45_49,anos_estudo,rendimento,alfabetizacao
0,Porto Velho,28.929758,10.144987,14.746582,11.198315,6.807736,650.66,92.290688
1,Rio Branco,31.230688,10.123926,12.617430,10.488707,5.842848,526.33,88.012278
2,Manaus,32.057934,10.030349,13.415472,10.998072,7.116276,592.34,94.109185
3,Boa Vista,28.063914,11.328753,15.034352,10.334746,6.739421,630.66,92.486446
4,Belém,35.375162,7.684457,11.740354,11.185196,7.435013,651.88,94.908148
5,Macapá,33.269868,11.868624,14.122916,10.002488,6.849520,671.75,91.360476
6,Palmas,23.380033,15.059800,15.084208,8.408592,7.548321,655.86,94.505742
7,São Luís,34.104069,8.625614,13.075440,11.026693,7.477652,549.91,93.324636
8,Teresina,31.417026,7.418865,12.422896,11.924180,6.308521,503.62,87.368525
9,Fortaleza,33.205659,6.944381,11.636174,11.058573,6.723686,648.13,89.851479


## Normalização

### Preparando características e alvos

In [14]:
rendimento_mediana = df['rendimento'].median()
df['classe_alvo'] = (df['rendimento'] > rendimento_mediana).astype(int)

print(f"Mediana do rendimento: R$ {rendimento_mediana:.2f}")
print("\nDistribuição das classes:")
print(df['classe_alvo'].value_counts())

caracteristicas = ['perc_mulheres', 'perc_25_29', 'perc_35_39', 'perc_45_49', 
           'anos_estudo', 'alfabetizacao']

X = df[caracteristicas].values
regressao_y = df['rendimento'].values
classificacao_y = df['classe_alvo'].values

print(f"\nShape dados: X {X.shape}, y {classificacao_y.shape}")

Mediana do rendimento: R$ 656.84

Distribuição das classes:
classe_alvo
0    10
1    10
Name: count, dtype: int64

Shape dados: X (20, 6), y (20,)


### Pré-Processamento

In [15]:
X_treino, X_teste, y_treino, y_teste = train_test_split(
    X, classificacao_y, test_size=0.2, random_state=42, stratify=classificacao_y
)

normalizador = StandardScaler()
X_treino_normalizado = normalizador.fit_transform(X_treino)
X_teste_normalizado = normalizador.transform(X_teste)

print(f"Treino: {X_treino_normalizado.shape}, Teste: {X_teste_normalizado.shape}")
print(f"Distribuição treino: {np.bincount(y_treino)}")
print(f"Distribuição teste: {np.bincount(y_teste)}")

Treino: (16, 6), Teste: (4, 6)
Distribuição treino: [8 8]
Distribuição teste: [2 2]
